In [11]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars
using JLD2
using Serialization

In [238]:
include("contractions.jl")
include("visualisation.jl")
include("preprocessing.jl")

loadsc (generic function with 1 method)

In [93]:
cubedata = loadsc("other_ply/cube.ply")
cube = initialContractedSimplicialComplex2D(cubedata)
visualize(cube)

0.0%┣                                               ┫ 0/12 [00:00<00:00, -0s/it]
8.3%┣███▋                                       ┫ 1/12 [00:00<Inf:Inf, InfGs/it]
100.0%┣███████████████████████████████████████████┫ 12/12 [00:00<00:00, 186it/s]
100.0%┣███████████████████████████████████████████┫ 12/12 [00:00<00:00, 185it/s]
┌ Warning: `shading = false` is not valid. Use `Makie.automatic`, `NoShading`, `FastShading` or `MultiLightShading`. Defaulting to `NoShading`.
└ @ Makie /home/yon/.julia/packages/Makie/pFPBw/src/lighting.jl:243
┌ Warning: `shading = false` is not valid. Use `Makie.automatic`, `NoShading`, `FastShading` or `MultiLightShading`. Defaulting to `NoShading`.
└ @ Makie /home/yon/.julia/packages/Makie/pFPBw/src/lighting.jl:243


GLMakie.Screen(...)

In [239]:
bunidata = deserialize("bunidata.dat")
buni = initialContractedSimplicialComplex2D(bunidata)
fillholes!(buni.contracted)
calculateFundamentalQuadratics!(buni)

In [204]:
marked_vertices = Set{Int}()
for edge in getborder(buni.original)
    push!(marked_vertices, edge[1])
    push!(marked_vertices, edge[2])
end
marks::Vector{GeometryBasics.Point{3,Float32}} = [GeometryBasics.Point{3,Float32}(buni.original.coords[x]) for x in marked_vertices]
visualize(buni, marks)

GLMakie.Screen(...)

In [240]:
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

0.0%┣                                           ┫ 0/104.5k [00:00<00:-8, -0s/it]
0.9%┣▍                                     ┫ 946/104.5k [00:00<00:08, 13.3kit/s]
3.1%┣█▏                                   ┫ 3.2k/104.5k [00:00<00:04, 26.4kit/s]
5.4%┣██                                   ┫ 5.7k/104.5k [00:00<00:03, 32.8kit/s]
7.6%┣██▉                                  ┫ 7.9k/104.5k [00:00<00:03, 35.6kit/s]
9.6%┣███▌                                ┫ 10.0k/104.5k [00:00<00:03, 36.5kit/s]
11.6%┣████                               ┫ 12.1k/104.5k [00:00<00:02, 37.3kit/s]
14.6%┣█████                              ┫ 15.2k/104.5k [00:00<00:02, 40.4kit/s]
17.6%┣██████▏                            ┫ 18.4k/104.5k [00:00<00:02, 43.2kit/s]
20.0%┣███████                            ┫ 20.9k/104.5k [00:00<00:02, 43.7kit/s]
22.1%┣███████▊                           ┫ 23.1k/104.5k [00:01<00:02, 43.7kit/s]
24.4%┣████████▌                          ┫ 25.5k/104.5k [00:01<00:02, 43.9kit/s]
26.9%┣█████████▍            

In [241]:
for vx in buni.contracted.vertices
    edgs = buni.contracted._vertex_to_edges[vx]
    triangls = Set(union([buni.contracted._edge_to_triangles[edge] for edge in edgs]...))
end

In [ ]:
edgs = Vector{Tuple{Int,Int}}()

(16236, 16421)

In [235]:
myedge = dequeue!(pq)
while !issafe(buni.contracted, myedge)
    myedge = dequeue!(pq)
end

new_vertex = contract!(buni, myedge)
for edge in buni.contracted._vertex_to_edges[new_vertex]
    pq[edge] = error(buni, edge)
end

In [ ]:
for _ in ProgressBar(1:3)
    myedge = dequeue!(pq)
    while !issafe(buni.contracted, myedge)
        myedge = dequeue!(pq)
    end

    new_vertex = contract!(buni, myedge)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
    end
end

0.0%┣                                                ┫ 0/3 [00:00<00:00, -0s/it]
100.0%┣█████████████████████████████████████████████┫ 3/3 [00:00<00:00, 600it/s]
100.0%┣█████████████████████████████████████████████┫ 3/3 [00:00<00:00, 528it/s]
